## Задача 1

In [1]:
import numpy as np
import itertools
import math
from collections import namedtuple
from graphviz import Digraph
from scipy.stats import norm
from random import random

## Ввод для задачи 1

In [2]:
H = np.array([
    [1, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0],
    [0, 0, 1, 0, 1, 1]
])

H_sym = np.array([
    [1, 0, 1, 1, 0, 0],
    [1, 1, 1, 0, 1, 0],
    [1, 1, 0, 0, 0, 1]
])

G = np.array([
    [1, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 1, 0]
])

G_MSF = np.array([
    [1, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 1, 0]
])
print "G * H^T: (must be zero, otherwise you suck)"
print np.mod(G.dot(H_sym.T), 2)

G * H^T: (must be zero, otherwise you suck)
[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [3]:
def gen_permutations_nz(n):
    res = [list(seq) for seq in itertools.product([0,1], repeat=n)]
    del res[0]

    return np.array(res)

def get_min(g):
    perm = gen_permutations_nz(g.shape[0])
    return np.min(np.sum(np.mod(np.dot(perm, g), 2), axis = 1))


print "MIN WEIGHT: {}".format(get_min(G))

MIN WEIGHT: 3


In [4]:
print "CODEWORDS:"
for codeword in np.mod(gen_permutations_nz(3).dot(G), 2):
    print "".join([str(i) for i in codeword])

CODEWORDS:
001110
010011
011101
100111
101001
110100
111010


In [5]:
def gen_permutations(n):
    res = [list(seq) for seq in itertools.product([0,1], repeat=n)]
    return np.array(res)


# Узел решетки        
Node = namedtuple("Node", "actives uid edges")   

ids = 0
def new_node(actives):
    global ids
    ids += 1
    return Node(actives, ids - 1, [])
  
def count_label(node):
    return "".join(map(str, node.actives.values()))

# Построим узлы решётки
g = G_MSF
begins = np.apply_along_axis(lambda row: np.min(np.flatnonzero(row)), 1, g)
ends   = np.apply_along_axis(lambda row: np.max(np.flatnonzero(row)), 1, g)

# print("Begin:", begins)
# print("End:  ", ends)

ids = 0
layers  = [[new_node(dict())]]
actives = [[]]
for i in range(0, g.shape[1]):
    layer  = []
    active = np.nonzero(np.logical_and(begins <= i, i < ends))[0]
    node_values = gen_permutations(len(active))
    for node_value in node_values:
        layer.append(new_node(dict(zip(active, node_value))))
    
    layers.append(layer)
    actives.append(active + 1)
    

# print()
# for i, layer in enumerate(layers):
#     print("Layer", i, "symbols", actives[i], "nodes", ", ".join(map(lambda n: "{} : {}".format(n.uid, count_label(n)), layer)))

# Построим ребра решётки
for i in range(len(layers) - 1):
#     print("Layer ", i)
    cur_layer = layers[i]
    nex_layer = layers[i + 1]
    for cur_node in cur_layer:
        for nex_node in nex_layer:
            isymbols = np.zeros(g.shape[0], np.int64) 
            mismatched = False
            for pos in range(len(isymbols)):
                value = cur_node.actives.get(pos)
                if value is None:
                    value = nex_node.actives.get(pos)
                    if value is None:
                        value = 0
                else:
                    nex_value = nex_node.actives.get(pos)
                    if nex_value is not None and nex_value != value:
                        mismatched = True
                        break
                isymbols[pos] = value
            if mismatched:
                continue
            c = np.mod(np.dot(g[:,i], isymbols) , 2)
#             print("{} --{}--> {}".format(cur_node.uid, c, nex_node.uid))
            cur_node.edges.append((c, nex_node))
    
# Построение графа с помощью утилиты graphviz 
dot = Digraph()
dot.graph_attr['rankdir'] = 'LR'
dot.graph_attr['nodesep'] = '0'
dot.graph_attr['ranksep'] = '1'
dot.graph_attr['splines'] = 'polyline'
for i, layer in enumerate(layers):
    with dot.subgraph(name='cluster_{}'.format(i)) as c:
        c.attr(label='layer #{}\nsymbols {}'.format(i, actives[i]), pencolor='transparent')
        for node in layer:
            label = count_label(node)
            c.node(str(node.uid), label)
            for edge in node.edges:
                dot.edge(str(node.uid), str(edge[1].uid), str(edge[0]))

dot.render('res_spen')  

'res_spen.pdf'

In [6]:
def generate(l):
    res = []
    
    for i in range(1, 2**l):
        b = bin(i)[2:]
        b = '0' * (l - len(b)) + b
        b = np.array(list(map(lambda x: int(x), b)))
        res.append(b)
        
    return np.array(res)

n = 6
k = 3
r = 3

a_s = generate(r)
a_e = generate(n)

def get_min(s):
    res = a_e.shape[1]
    for e in a_e:
        if np.all(s == (np.dot(e, H.T) % 2)):
            res = min(res, np.sum(e))
    return res


def get_syndrom_matrix():
    res = []
    for s in a_s:
        m = get_min(s)
        
        for e in a_e:
            if np.all(s == (np.dot(e, H.T) % 2)) and (np.sum(e) == m):
                res.append((s, e))
                break
    
    return res

mat = get_syndrom_matrix()
print "SYNDROMES TABLE:"
for l in mat:
    print("".join([str(i) for i in l[0]]), "".join([str(i) for i in l[1]]))

SYNDROMES TABLE:
('001', '000001')
('010', '010000')
('011', '000010')
('100', '100000')
('101', '001000')
('110', '000100')
('111', '000101')


## Задача 2

## Ввод для задачи 2

In [7]:
N = 31
D = 6
FIELD_POLY = [1, 0, 1, 0, 0, 1]
ALPHA = [0, 0, 0, 0, 1]
MIN_POLYNOMES = [0, 1, 3]

ERROR_COUNT = 2

In [8]:
# Count cyclotomic classes

c_classes = {}

def init_c_class(i, n):
        c_class = [i]
        while True:
            nxt = (c_class[-1] * 2) % n
            if not nxt in c_class:
                c_class.append(nxt)
            else:
                break
        return c_class

used = [False for i in range(N)]
for i in range(N):
    if used[i] == False:
        c_class = init_c_class(i, N)
        for val in c_class:
            used[val] = True
        c_classes[i] = c_class

print c_classes
        
def add_mod(a, b):
    for i in range(len(a)):
        a[i] = (a[i] + b[i]) % 2
    return a

class GaloisField(object):
    def __init__(self, poly):
        self.n = 2 ** (len(poly) - 1)
        self.mod = poly[1:]
        self.pow_to_poly = {}
        self.pow_to_poly[-1] = np.zeros(len(self.mod))
        one = np.zeros(len(self.mod))
        one[-1] = 1
        self.pow_to_poly[0] = one
    
    def build(self, gen_poly):
        for i in range(1, self.n-1):
            c = self.mul(self.pow_to_poly[i-1], gen_poly)
            self.pow_to_poly[i] = c
    
    def mul(self, a, b):
        c = np.zeros(len(a))
        for i in reversed(range(len(b))):
            for j in reversed(range(len(a))):
                if b[i] == 1 and a[j] == 1:
                    k = len(c) - len(b) - len(a) + i + j
                    if k >= 0:
                        c[k] = (c[k] + a[j]) % 2
                    else:
                        add_mod(c, self.mod)
        return c
    
    def get_power(self, a):
        for k, v in self.pow_to_poly.items():
            if np.array_equal(v, a):
                return k
        return -2
    
gf = GaloisField(FIELD_POLY)
gf.build(ALPHA)
print "GALUA FIELD:"
for k, v in gf.pow_to_poly.items():
    print(k, "".join([str(int(i)) for i in v]))

{0: [0], 1: [1, 2, 4, 8, 16], 3: [3, 6, 12, 24, 17], 5: [5, 10, 20, 9, 18], 7: [7, 14, 28, 25, 19], 11: [11, 22, 13, 26, 21], 15: [15, 30, 29, 27, 23]}
GALUA FIELD:
(0, '00001')
(1, '00010')
(2, '00100')
(3, '01000')
(4, '10000')
(5, '01001')
(6, '10010')
(7, '01101')
(8, '11010')
(9, '11101')
(10, '10011')
(11, '01111')
(12, '11110')
(13, '10101')
(14, '00011')
(15, '00110')
(16, '01100')
(17, '11000')
(18, '11001')
(19, '11011')
(20, '11111')
(21, '10111')
(22, '00111')
(23, '01110')
(24, '11100')
(25, '10001')
(26, '01011')
(27, '10110')
(28, '00101')
(29, '01010')
(30, '10100')
(-1, '00000')


In [9]:
stack = []

def mult(a, b):
    res = []
    for i in range(len(a) + len(b)):
        res.append([])
    for idx_a, pows_a in enumerate(a):
        for idx_b, pows_b in enumerate(b):
            for i_a in pows_a:
                for i_b in pows_b:
                    res[idx_a + idx_b].append(i_a + i_b)
    return res
                    
    
def build_gen_poly(indexes):
    res = []
    for ind in indexes:
        c_class = c_classes[ind]
        for val in c_class:
            stack.append([[val], [0]])
    while not len(stack) == 1:
        stack.append(mult(stack.pop(), stack.pop()));
    coef = stack[0]
    size = len(gf.mod)
    for idx, val in enumerate(coef):
        a = np.zeros(size)
        for pow in val:
            add_mod(a, gf.pow_to_poly[pow % N])
        if any(a_val != 0 for a_val in a):
            res.append(idx)
    return res
gen_poly = build_gen_poly(MIN_POLYNOMES)
print "GENERATING POLYNOME:"
print gen_poly

GENERATING POLYNOME:
[0, 3, 4, 6, 7, 8, 10, 11]


In [10]:
WORD = list(gen_poly)

for i in range(ERROR_COUNT):
    if i in WORD:
        WORD.remove(i)
    else:
        WORD.insert(0, i)
print "SPOILED WORD:"
print WORD

SPOILED WORD:
[1, 3, 4, 6, 7, 8, 10, 11]


In [11]:
def ind_to_pow(i, field, powers, n):
    size = len(field.mod)
    a = np.zeros(size)
    for pow in powers:
        add_mod(a, field.pow_to_poly[(pow * i) % n])
    for k, v in field.pow_to_poly.items():
        if np.array_equal(v, a):
            return k
    return -2


def build_synd_poly(field, n, d, powers):
    return map(lambda x: ind_to_pow(x, field, powers, n), range(1, d))

synd_poly = build_synd_poly(gf, N, D, WORD)
print "SYNDROME POLYNOME:"
print synd_poly

SYNDROME POLYNOME:
[14, 28, 5, 25, 23]


In [12]:
B = synd_poly[ERROR_COUNT: 2 * ERROR_COUNT]
A = []
for i in range(ERROR_COUNT):
    A.append(synd_poly[i:i + ERROR_COUNT])

In [13]:
def copy_add_mod(a, b):
    tmp = []
    for i in range(len(a)):
        tmp.append((a[i] + b[i]) % 2)
    return tmp

def solve(gf, n, a, b):
    for i in range(len(a)):
        pow = n - a[i][i]
        for j in range(i, len(a[i])):
            a[i][j] = (a[i][j] + pow) % n
        b[i] = (b[i] + pow) % n
        for j in range(i + 1, len(a)):
            diff_pow = a[j][i] - a[i][i]
            for k in range(i, len(a[j])):
                add1 = gf.pow_to_poly[(a[i][k] + diff_pow) % n]
                add2 = gf.pow_to_poly[a[j][k]]
                tmp = copy_add_mod(add1, add2)
                a[j][k] = gf.get_power(tmp)
            add1 = gf.pow_to_poly[(b[i] + diff_pow) % n]
            add2 = gf.pow_to_poly[b[j]]
            tmp = copy_add_mod(add1, add2)
            b[j] = gf.get_power(tmp)
    
    for i in reversed(range(len(a))):
        for j in range(i):
            pow = a[j][i]
            a[j][i] = -1
            tmp = copy_add_mod(gf.pow_to_poly[(b[i] + pow) % n], gf.pow_to_poly[b[j]])
            b[j] = gf.get_power(tmp)
    return b

sol = solve(gf, N, A, B)
sol = list(reversed(sol + [0]))
print "LOCATOR POLYNOME COEFFS:"
print sol

LOCATOR POLYNOME COEFFS:
[0, 14, 1]


In [14]:
def is_root(x, gf, coefs, n):
    res = np.zeros(len(gf.mod))
    for j, val in enumerate(coefs):
        if val != -1:
            add_mod(res, gf.pow_to_poly[(val + x * j) % n])
    return all(r == 0 for r in res)
    

def find_locator_error(gf, n, coefs):
    tmp = filter(lambda x: is_root(x, gf, coefs, n), range(n))
    return map(lambda i: (n - i) % n, tmp)

locator_errors = find_locator_error(gf, N, sol)
print "ERROR POSITON:"
print locator_errors

ERROR POSITON:
[0, 1]


In [15]:
B = synd_poly[:ERROR_COUNT]
A = []
for i in range(ERROR_COUNT):
    A.append([])
    for j in range(ERROR_COUNT):
        A[i].append(locator_errors[j] * (i + 1) % N)
error_values = solve(gf, N, A, B)
print "ERROR VALUES:"
print error_values

ERROR VALUES:
[0, 0]


In [20]:
range_i = MIN_POLYNOMES

def print_H(range_i):
    H = [[] for i in range_i]
    for idx, i in enumerate(range_i):
        for j in range(N):
            cur_deg = gf.pow_to_poly[j * (i) % N]
            pow = gf.get_power(cur_deg)
            H[idx].append(cur_deg)
    for row in H:
        for bin_row in reversed(list(np.transpose(row))):
            s = "".join([str(int(i)) for i in bin_row])
            if 1 in bin_row:
                print s
            else:
                print s + "<-- спорно; зависит от ответа Кудряшова"
            
print "H(version_1)"
print_H(range_i)

H(version_1)
1111111111111111111111111111111
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
1000010101110110001111100110100
0100001010111011000111110011010
0010000101011101100011111001101
0001010111011000111110011010010
0000101011101100011111001101001
1001001100001011010100011101111
0010110101000111011111001001100
0001110111110010011000010110101
0101101010001110111110010011000
0011101111100100110000101101010


In [21]:
range_i = range(MIN_POLYNOMES[0], MIN_POLYNOMES[0] + D - 2)

print "H(version_2)"
print_H(range_i)

H(version_2)
1111111111111111111111111111111
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
0000000000000000000000000000000<-- спорно; зависит от ответа Кудряшова
1000010101110110001111100110100
0100001010111011000111110011010
0010000101011101100011111001101
0001010111011000111110011010010
0000101011101100011111001101001
1000010101110110001111100110100
0001111100110100100001010111011
0100001010111011000111110011010
0000101011101100011111001101001
0011111001101001000010101110110
1001001100001011010100011101111
0010110101000111011111001001100
0001110111110010011000010110101
0101101010001110111110010011000
0011101111100100110000101101010
